In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import pandas as pd

In [8]:
job_name = "Machine Learning"
country_name = "Santiago Metropolitan Region Chile"
job_url ="";
for item in job_name.split(" "):
    if item != job_name.split(" ")[-1]:
        job_url = job_url + item + "%20"
    else:
        job_url = job_url + item

country_url ="";
for item in country_name.split(" "):
    if item != country_name.split(" ")[-1]:
        country_url = country_url + item + "%20"
    else:
        country_url = country_url + item

url = f"https://www.linkedin.com/jobs/search?keywords={job_url}&location={country_url}&geoId=104555257&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0"
# Creating a webdriver instance
options = webdriver.ChromeOptions()
#All are optional
# Agrega la opcion de que se quede abierta la pagina
# options.add_experimental_option("detach", True)
options.add_experimental_option('excludeSwitches', ['enable-logging'])
options.add_argument("--disable-extensions")
options.add_argument("--disable-notifications")
options.add_argument("--disable-Advertisement")
options.add_argument("--disable-popup-blocking")
options.add_argument("start-maximized")

driver= webdriver.Chrome("chromedrive",options = options)
# Opening the url we have just defined in our browser
driver.get(url)
print(url)

C:\Users\raulv\AppData\Local\Temp\ipykernel_13872\986121327.py:30: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver= webdriver.Chrome("chromedrive",options = options)


https://www.linkedin.com/jobs/search?keywords=Machine%20Learning&location=Santiago%20Metropolitan%20Region%20Chile&geoId=104555257&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0


In [11]:
#We find how many jobs are offered.
jobs_num = driver.find_element(By.CSS_SELECTOR,"h1>span").get_attribute("innerText")
if len(jobs_num.split(',')) > 1:
    jobs_num = int(jobs_num.split(',')[0])*1000
else:
    jobs_num = int(jobs_num)

jobs_num   = int(jobs_num)
print(jobs_num)


236


In [13]:
#We create a while loop to browse all jobs. 
i = 2
while i <= int(jobs_num/2)+1:
    #We keep scrollind down to the end of the view.
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    i = i + 1
    print("Current at: ", i, "Percentage at: ", ((i+1)/(int(jobs_num/2)+1))*100, "%",end="\r")
    try:
        #We try to click on the load more results buttons in case it is already displayed.
        infinite_scroller_button = driver.find_element(By.XPATH, ".//button[@aria-label='See more jobs']")
        infinite_scroller_button.click()
        time.sleep(2)
    except:
        #If there is no button, there will be an error, so we keep scrolling down.
        time.sleep(0.1)
        pass
#We get a list containing all jobs that we have found.
job_lists = driver.find_element(By.CLASS_NAME,"jobs-search__results-list")
jobs = job_lists.find_elements(By.TAG_NAME,"li") # return a list

In [14]:
#We declare void list to keep track of all obtaind data.
job_title_list = []
company_name_list = []
location_list = []
date_list = []
job_link_list = []

#We loof over every job and obtain all the wanted info.
for job in jobs:
    #job_title
    job_title = job.find_element(By.CSS_SELECTOR,"h3").get_attribute("innerText")
    job_title_list.append(job_title)
    
    #company_name
    company_name = job.find_element(By.CSS_SELECTOR,"h4").get_attribute("innerText")
    company_name_list.append(company_name)
    
    #location
    location = job.find_element(By.CSS_SELECTOR,"div>div>span").get_attribute("innerText")
    location_list.append(location)
    
    #date
    date = job.find_element(By.CSS_SELECTOR,"div>div>time").get_attribute("datetime")
    date_list.append(date)
    
    #job_link
    job_link = job.find_element(By.CSS_SELECTOR,"a").get_attribute("href")
    job_link_list.append(job_link)

In [15]:
df = pd.DataFrame(list(zip(job_title_list,company_name_list,
 location_list,job_link_list,
 date_list)),
 columns =['job_title', 'company_name',
 'company_location','Job_link',
 'post_date'])
# Storing the data to csv file
df.to_csv('job_offers.csv', index=False)